In [37]:
import requests
import pandas as pd
from random_word import Wordnik
import time
import math

In [62]:
def generate_search_strings(num_items: int):    
    batch_size = 10
    num_calls = math.ceil(num_items/batch_size)
    output = []
    wordnik_service = Wordnik()
    for i in range(num_calls):
        # Return a single random word
        res = wordnik_service.get_random_words(includePartOfSpeech ="noun,verb,adverb",hasDictionaryDef=True, limit=batch_size)
        #TODO: Check if adding a duplicate search term
        [output.append(x) for x in res]
    return output

def lexica_search(terms: list):
    search_base='https://lexica.art/api/v1/search?q='
    prompts = pd.DataFrame(columns=['source','prompt'])
    for item in terms:
        print('Searching term: ' , item)
        query = item
        query = query.replace(' ', '+')
        try:
            d = requests.get(url=(search_base + query))
            data = d.json()
            obj = data['images']
        except:
            print("API Rate limited!")
            return prompts
        print('Adding items to db for search term: ', item)
        for item in obj:
            row = [item['src'],item['prompt']]
            prompts.loc[item['id']] = row
        time.sleep(3)
    return prompts

terms = generate_search_strings(num_items=5)
res = lexica_search(terms=terms)
res.head()


Searching term:  hiver
Adding items to db for search term:  hiver
Searching term:  swillings
Adding items to db for search term:  swillings
Searching term:  repositor
Adding items to db for search term:  repositor
Searching term:  sportiveness
Adding items to db for search term:  sportiveness
Searching term:  discontinuance
Adding items to db for search term:  discontinuance
Searching term:  colourization
Adding items to db for search term:  colourization
Searching term:  Orozco
Adding items to db for search term:  Orozco
Searching term:  jack-fool
Adding items to db for search term:  jack-fool
Searching term:  cochairs
Adding items to db for search term:  cochairs
Searching term:  venge
Adding items to db for search term:  venge


,source,prompt
040942f3-0b15-49e2-866a-ef5873e92c9d,https://lexica-serve-encoded-images.sharif.wor...,portrait focus of knight beautiful 3 d anime g...
0b2d0850-e29d-4917-8fe7-334a3c929e5c,https://lexica-serve-encoded-images.sharif.wor...,a chibi kawaii red haired girl with winter clo...
102a6044-bc8c-4d95-8d35-0cd66a41b280,https://lexica-serve-encoded-images.sharif.wor...,a instax photo of a petite japanese girl in a ...
15e35bd1-1754-4e28-a9c3-7d6987310db7,https://lexica-serve-encoded-images.sharif.wor...,"walking amongst the stars, concept art, Kyoto ..."
1a15ed51-020f-4b5f-b39c-406a33c0096c,https://lexica-serve-encoded-images.sharif.wor...,Portrait of a white haired anime girl wearing ...


In [54]:
search_base='https://lexica.art/api/v1/search?q='
query='in the style of'
query = query.replace(' ', '+')
d = requests.get(url=(search_base + query))
data = d.json()
obj = data['images']
images = pd.DataFrame(columns=['source','prompt'])
for item in obj:
    row = [item['src'],item['prompt']]
    images.loc[item['id']] = row


(50, 2)

In [31]:
wordnik_service = Wordnik()
res = wordnik_service.get_random_words(includePartOfSpeech ="noun,verb,adverb",hasDictionaryDef=True, limit=50)

type(res)

list